# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 23 2022 2:45PM,237905,20,SSNA-2173624969,Sartorius Stedim North America,Released
1,May 23 2022 2:43PM,237904,21,26694095,"NBTY Global, Inc.",Released
2,May 23 2022 2:41PM,237903,20,SSNA-2173625080,Sartorius Stedim North America,Released
3,May 23 2022 2:41PM,237902,10,SONSB0001796,"Nextsource Biotechnology, LLC",Released
4,May 23 2022 2:37PM,237899,18,HH-34491,Hush Hush,Released
5,May 23 2022 2:37PM,237899,18,HH-34492,Hush Hush,Released
6,May 23 2022 2:37PM,237899,18,HH-34494,Hush Hush,Released
7,May 23 2022 2:37PM,237899,18,HH-34495,Hush Hush,Released
8,May 23 2022 2:37PM,237899,18,HH-34496,Hush Hush,Released
9,May 23 2022 2:36PM,237898,18,HH-34493,Hush Hush,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
47,237902,Released,1
48,237903,Released,1
49,237904,Released,1
50,237905,Released,1
51,237907,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
237902,NaN,NaN,1.0
237903,NaN,NaN,1.0
237904,NaN,NaN,1.0
237905,NaN,NaN,1.0
237907,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237673,0.0,0.0,1.0
237701,0.0,0.0,1.0
237794,3.0,13.0,10.0
237804,12.0,7.0,0.0
237808,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237673,0,0,1
237701,0,0,1
237794,3,13,10
237804,12,7,0
237808,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237673,0,0,1
1,237701,0,0,1
2,237794,3,13,10
3,237804,12,7,0
4,237808,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237673,,,1
1,237701,,,1
2,237794,3,13,10
3,237804,12,7,
4,237808,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 23 2022 2:45PM,237905,20,Sartorius Stedim North America
1,May 23 2022 2:43PM,237904,21,"NBTY Global, Inc."
2,May 23 2022 2:41PM,237903,20,Sartorius Stedim North America
3,May 23 2022 2:41PM,237902,10,"Nextsource Biotechnology, LLC"
4,May 23 2022 2:37PM,237899,18,Hush Hush
9,May 23 2022 2:36PM,237898,18,Hush Hush
10,May 23 2022 2:35PM,237701,18,Hush Hush
11,May 23 2022 2:27PM,237897,20,Sartorius Stedim North America
12,May 23 2022 2:24PM,237896,21,"NBTY Global, Inc."
13,May 23 2022 2:23PM,237895,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 23 2022 2:45PM,237905,20,Sartorius Stedim North America,,,1
1,May 23 2022 2:43PM,237904,21,"NBTY Global, Inc.",,,1
2,May 23 2022 2:41PM,237903,20,Sartorius Stedim North America,,,1
3,May 23 2022 2:41PM,237902,10,"Nextsource Biotechnology, LLC",,,1
4,May 23 2022 2:37PM,237899,18,Hush Hush,,,5
5,May 23 2022 2:36PM,237898,18,Hush Hush,,,1
6,May 23 2022 2:35PM,237701,18,Hush Hush,,,1
7,May 23 2022 2:27PM,237897,20,Sartorius Stedim North America,,,1
8,May 23 2022 2:24PM,237896,21,"NBTY Global, Inc.",,,1
9,May 23 2022 2:23PM,237895,21,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2022 2:45PM,237905,20,Sartorius Stedim North America,1,,
1,May 23 2022 2:43PM,237904,21,"NBTY Global, Inc.",1,,
2,May 23 2022 2:41PM,237903,20,Sartorius Stedim North America,1,,
3,May 23 2022 2:41PM,237902,10,"Nextsource Biotechnology, LLC",1,,
4,May 23 2022 2:37PM,237899,18,Hush Hush,5,,
5,May 23 2022 2:36PM,237898,18,Hush Hush,1,,
6,May 23 2022 2:35PM,237701,18,Hush Hush,1,,
7,May 23 2022 2:27PM,237897,20,Sartorius Stedim North America,1,,
8,May 23 2022 2:24PM,237896,21,"NBTY Global, Inc.",1,,
9,May 23 2022 2:23PM,237895,21,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2022 2:45PM,237905,20,Sartorius Stedim North America,1,,
1,May 23 2022 2:43PM,237904,21,"NBTY Global, Inc.",1,,
2,May 23 2022 2:41PM,237903,20,Sartorius Stedim North America,1,,
3,May 23 2022 2:41PM,237902,10,"Nextsource Biotechnology, LLC",1,,
4,May 23 2022 2:37PM,237899,18,Hush Hush,5,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2022 2:45PM,237905,20,Sartorius Stedim North America,1.0,NaN,NaN
1,May 23 2022 2:43PM,237904,21,"NBTY Global, Inc.",1.0,NaN,NaN
2,May 23 2022 2:41PM,237903,20,Sartorius Stedim North America,1.0,NaN,NaN
3,May 23 2022 2:41PM,237902,10,"Nextsource Biotechnology, LLC",1.0,NaN,NaN
4,May 23 2022 2:37PM,237899,18,Hush Hush,5.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2022 2:45PM,237905,20,Sartorius Stedim North America,1.0,0.0,0.0
1,May 23 2022 2:43PM,237904,21,"NBTY Global, Inc.",1.0,0.0,0.0
2,May 23 2022 2:41PM,237903,20,Sartorius Stedim North America,1.0,0.0,0.0
3,May 23 2022 2:41PM,237902,10,"Nextsource Biotechnology, LLC",1.0,0.0,0.0
4,May 23 2022 2:37PM,237899,18,Hush Hush,5.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2378571,153.0,22.0,12.0
12,713583,3.0,1.0,0.0
15,237794,10.0,13.0,3.0
16,475771,2.0,0.0,0.0
18,1902448,12.0,0.0,0.0
19,951522,20.0,4.0,0.0
20,2140928,77.0,13.0,17.0
21,713695,3.0,0.0,0.0
22,713640,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2378571,153.0,22.0,12.0
1,12,713583,3.0,1.0,0.0
2,15,237794,10.0,13.0,3.0
3,16,475771,2.0,0.0,0.0
4,18,1902448,12.0,0.0,0.0
5,19,951522,20.0,4.0,0.0
6,20,2140928,77.0,13.0,17.0
7,21,713695,3.0,0.0,0.0
8,22,713640,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,153.0,22.0,12.0
1,12,3.0,1.0,0.0
2,15,10.0,13.0,3.0
3,16,2.0,0.0,0.0
4,18,12.0,0.0,0.0
5,19,20.0,4.0,0.0
6,20,77.0,13.0,17.0
7,21,3.0,0.0,0.0
8,22,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,153.0
1,12,Released,3.0
2,15,Released,10.0
3,16,Released,2.0
4,18,Released,12.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20,21,22
Status,,,,,,,,,
Completed,12.0,0.0,3.0,0.0,0.0,0.0,17.0,0.0,0.0
Executing,22.0,1.0,13.0,0.0,0.0,4.0,13.0,0.0,0.0
Released,153.0,3.0,10.0,2.0,12.0,20.0,77.0,3.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20,21,22
0,Completed,12.0,0.0,3.0,0.0,0.0,0.0,17.0,0.0,0.0
1,Executing,22.0,1.0,13.0,0.0,0.0,4.0,13.0,0.0,0.0
2,Released,153.0,3.0,10.0,2.0,12.0,20.0,77.0,3.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20,21,22
0,Completed,12.0,0.0,3.0,0.0,0.0,0.0,17.0,0.0,0.0
1,Executing,22.0,1.0,13.0,0.0,0.0,4.0,13.0,0.0,0.0
2,Released,153.0,3.0,10.0,2.0,12.0,20.0,77.0,3.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()